In this notebook, we'll prepare the code for all transformers, so that we'll see whether the hyperparameters (especially the max_seq_length parameter which causes errors) work on all of them.

Import all necessary libraries and install everything you need for training:

In [1]:
# install pytorch
!conda install --yes pytorch>=1.6 cudatoolkit=11.0 -c pytorch

In [2]:
# install simpletransformers
!pip install -q transformers
!pip install --upgrade transformers
!pip install -q simpletransformers

# check installed version
!pip freeze | grep simpletransformers

In [3]:
# install stable torch
!pip uninstall -q torch -y
!pip install -q torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [4]:
# install the libraries necessary for data wrangling, prediction and result analysis
from sklearn.metrics import f1_score, ConfusionMatrixDisplay, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Import the data

In [5]:
# Import the data, prepared for the experiments
train_df = pd.read_csv("/kaggle/input/gincodataframededuptraindevtest/GINCO_dataframe_dedup_train_dev.csv")
test_df = pd.read_csv("/kaggle/input/gincodataframededuptraindevtest/GINCO_dataframe_dedup_test.csv")

print("Train shape: {}, Test shape: {}.".format(train_df.shape, test_df.shape))

In [6]:
# Create a list of labels
LABELS = train_df.labels.unique().tolist()

In [7]:
# Drop the instances with no text
train_df = train_df.dropna()
test_df = test_df.dropna()
print("Train shape: {}, Test shape: {}.".format(train_df.shape, test_df.shape))

## Transformers

Let's start with arguments which are the same for all the models.

In [8]:
# define hyperparameters
model_args ={"overwrite_output_dir": True,
             "num_train_epochs": 90,
             "labels_list": LABELS,
             "learning_rate": 1e-5,
             "train_batch_size": 32,
             "no_cache": True,
             "no_save": True,
             "max_seq_length": 300,
             "save_steps": -1,
             }

In [9]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

### SloBERTa
Slovene model
https://huggingface.co/EMBEDDIA/sloberta

from simpletransformers.classification import ClassificationModel

sloberta_model = ClassificationModel(
    "camembert", "EMBEDDIA/sloberta",
    use_cuda = True,
    num_labels = 21,
    args = model_args)

### CroSloEngual BERT
Slovene-Croatian-English model
https://huggingface.co/EMBEDDIA/crosloengual-bert

from simpletransformers.classification import ClassificationModel

crosloengualbert_model = ClassificationModel(
        "bert", "EMBEDDIA/crosloengual-bert",
        num_labels=21,
        use_cuda=True,
        args=model_args
    )

### Base-sized XLM-RoBERTa

Multilingual model
https://huggingface.co/xlm-roberta-base

from simpletransformers.classification import ClassificationModel

roberta_base_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-base",
        num_labels=21,
        use_cuda=True,
        args=model_args
    )

### Large-sized XML-RoBERTa
Multilingual model https://huggingface.co/xlm-roberta-large

from simpletransformers.classification import ClassificationModel

roberta_large_model = ClassificationModel(
        "xlmroberta", "xlm-roberta-large",
        num_labels=21,
        use_cuda=True,
        args=model_args
    )

### DeBERTaV3
Multilingual model https://huggingface.co/microsoft/mdeberta-v3-base

from simpletransformers.classification import ClassificationModel

debertav3_model = ClassificationModel(
        "debertav2", "microsoft/mdeberta-v3-base",
        num_labels=21,
        use_cuda=True,
        args=model_args
    )

### BERTić
Model for related South Slavic languages https://huggingface.co/classla/bcms-bertic

from simpletransformers.classification import ClassificationModel

bertic_model = ClassificationModel(
        "electra", "classla/bcms-bertic",
        num_labels=21,
        use_cuda=True,
        args=model_args
    )

### BERT base model (cased)
Monolingual English model https://huggingface.co/bert-base-cased

In [10]:
from simpletransformers.classification import ClassificationModel

bertbase_model = ClassificationModel(
        "bert", "bert-base-cased",
        num_labels=21,
        use_cuda=True,
        args=model_args
    )

## Training and evaluation

### Train

In [11]:
# SloBERTa
#sloberta_model.train_model(train_df)

# CroSloEngual BERT
#crosloengualbert_model.train_model(train_df)

# Base-sized XML-Roberta
#roberta_base_model.train_model(train_df)

# Large-sized XML-Roberta
#roberta_large_model.train_model(train_df)

# DeBERTav3
#debertav3_model.train_model(train_df)

# BERTić
#bertic_model.train_model(train_df)

# English base-sized BERT
bertbase_model.train_model(train_df)

### Evaluate

In [ ]:
def eval_model(model,plot_title=None):
    """ Evaluates the model. It takes the test data, named as "test_df", and labels list named "LABELS".
    
    Args: 
        model (simpletransformers.ClassificationModel): the model name.
        plot_title (string): the title of the confusion matrix, defaults to None.
    
    Returns:
    
    """
    instance_predictions, raw_outputs = sloberta_model.predict(['Danes poročamo o dogodku, ki se je zgodil 1. 1. 2020. Oseba je dejala:"To je res nenormalen dogodek"'])
    print("Instance prediction: ", instance_predictions)
    
    # Get the true labels from the dataframe
    y_true = test_df.labels

    # Calculate the model's predictions
    y_pred = model.predict(test_df.text.tolist())[0]
    
    macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
    micro = f1_score(y_true, y_pred, labels=LABELS,  average="micro")
    print(f"Macro f1: {macro:0.3}\nMicro f1: {micro:0.3}")
    
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
    classNames = labels
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)

    metrics = f"{microF1:0.4}, {macroF1:0.4}"
    if title:
        plt.title(title +";\n" + metrics)
    else:
        plt.title(metrics)
    plt.tight_layout()
    plt.show()
    return microF1, macroF1

Choose from the following models:
sloberta_model(train_df), crosloengualbert_model(train_df), roberta_base_model(train_df), roberta_large_model(train_df), debertav3_model(train_df), bertic_model(train_df), bertbase_model(train_df)

In [ ]:
eval_model(crosloengualbert,title="CroSloEngualBert-300-Initial_Setup")
#plt.savefig("SloBERTa-Initial-Setup_test_dev.png")

Produce the confusion matrix:

In [ ]:
def plot_cm(y_true, y_pred, labels, title=None):
    cm = confusion_matrix(y_true, y_pred, labels=labels, )
    # print(cm)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, '{:d}'.format(z), ha='center', va='center')
    classNames = labels
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    microF1 = f1_score(y_true, y_pred, labels=labels, average ="micro")
    macroF1 = f1_score(y_true, y_pred, labels=labels, average ="macro")

    print(f"Micro F1: {microF1:0.3}",f"Macro F1: {macroF1:0.3}")

    metrics = f"{microF1:0.4}, {macroF1:0.4}"
    if title:
        plt.title(title +";\n" + metrics)
    else:
        plt.title(metrics)
    plt.tight_layout()
    plt.show()
    return microF1, macroF1